In [1]:
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
#from glmnet import glmnet; from glmnetPlot import glmnetPlot 
#from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
#from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
#from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict
import glob
import csv
import numpy as np
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
%matplotlib inline
import matplotlib.pyplot as plt
from random import shuffle

In [2]:
files = (glob.glob("../../ovarian_cancer_results/collagen_final/features/*"))
shuffle(files)
print(len(files))

90


In [ ]:
with open('../notebooks/results_good/main_filenames.txt') as f:
    lines = f.readlines()
files = []
for line in lines:
    files.append(line[:len(line)-1])
print(files)

In [ ]:
#print(files)
#textfile = open("filenames.txt", "w")
#for file in files:
#    textfile.write(file + "\n")
#textfile.close()

In [3]:
collagen_features = []
for file in files:
    flag = -1
    file_features = []
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                for index in range(0, len(array)):
                    file_features.append(float(array[index]))
    collagen_features.append(file_features)
print(len(collagen_features))

90


In [4]:
f = []
flag = -1
with open("../../til_biomarkers_ovarian_cancer/data/features.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if flag == -1:
            flag = 1
        else:
            array = row
            ff = []
            for index in range(1, len(array)):
                ff.append(float(array[index]))
            f.append(ff)

til_features = []
for file in files:
    flag = -1
    count = 0
    target_file = file.split("/")[-1]
    with open("../../til_biomarkers_ovarian_cancer/data/filenames.csv", newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                file = array[1]
                file = file.split("/")[-1]
                
                if target_file == file:
                    til_features.append(f[count])
                count += 1
print(len(til_features))

90


In [5]:
features = []
for index in range(0, 90):
    #features.append(til_features[index]+collagen_features[index])
    features.append(collagen_features[index])
print(len(features))
print(len(features[0]))

90
72


In [6]:
print(len(features[0]))

72


In [7]:
train_features = []
val_features = []
count = 0
for index in range(30, 90):
    count += 1
    current_features = []
    current_features.append(count)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    train_features.append(current_features)

'''
for index in range(60, 90):
    count += 1
    current_features = []
    current_features.append(count)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    train_features.append(current_features)
'''
"""
for index in range(0, 27, 2):
    count += 1
    current_features = []
    current_features.append(count)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    train_features.append(current_features)
"""
    
count = 0
for index in range(0, 30):
    count += 1
    current_features = []
    current_features.append(count)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    val_features.append(current_features)

In [8]:
print(len(train_features))
print(len(val_features))

60
30


In [9]:
print(len(train_features[0]))
print(len(val_features[0]))

73
73


In [10]:
rows = []
rows.append("")
for index in range(1, 73):
    rows.append("F_" + str(index))
print(len(rows))

73


In [11]:
final_train_features = []
final_train_features.append(rows)
for index in range(0, len(train_features)):
    final_train_features.append(train_features[index])
print(len(final_train_features))

61


In [12]:
final_val_features = []
final_val_features.append(rows)
for index in range(0, len(val_features)):
    final_val_features.append(val_features[index])
print(len(final_val_features))

31


In [13]:
# write csv file
with open("train_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_features)):
        spamwriter.writerow(final_train_features[index])

In [14]:
# write csv file
with open("val_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_features)):
        spamwriter.writerow(final_val_features[index])

In [15]:
flag = -1
new_filenames = []
survival_info = []
age_info = []
with open("../../Ov_TCGA_data.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        file_survival_info = []
        if flag == -1:
            flag = 1
        else:
            array = row
            new_filenames.append(array[1])
            
            if array[7] == "--":
                array[7] = 20000
            
            if array[10] == "--":
                if array[len(array)-1] == "--":
                    array[len(array)-1] = 50
                file_survival_info.append(float(array[len(array)-1]))
                file_survival_info.append(0)
                age_info.append(float(array[7]))
            else:
                file_survival_info.append(float(array[10]))
                file_survival_info.append(1)
                age_info.append(float(array[7]))
            survival_info.append(file_survival_info)

In [16]:
import pandas as pd
df = pd.read_csv("../../clinical.tsv", sep='\t')
df.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,74dcdfb4-a637-4111-9a22-4b4bc7bbc8f7,TCGA-13-1495,TCGA-OV,60,'--,'--,'--,'--,-22046,2749,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"
1,74dcdfb4-a637-4111-9a22-4b4bc7bbc8f7,TCGA-13-1495,TCGA-OV,60,'--,'--,'--,'--,-22046,2749,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
2,f2ec64f5-a414-4c34-99c7-d59bc4e831e0,TCGA-24-1545,TCGA-OV,69,'--,'--,'--,'--,-25460,1746,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
3,f2ec64f5-a414-4c34-99c7-d59bc4e831e0,TCGA-24-1545,TCGA-OV,69,'--,'--,'--,'--,-25460,1746,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"
4,6fb71a0c-bc50-48c8-acfa-db94be1e151a,TCGA-29-1693,TCGA-OV,72,'--,'--,'--,'--,-26300,'--,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Radiation Therapy, NOS"


In [17]:
df = df[['figo_stage', 'case_submitter_id']]

In [18]:
df['Status'] = df['case_submitter_id'].isin(f)

In [19]:
df = df[df['Status'] == True]
df

,figo_stage,case_submitter_id,Status


In [20]:
for index, row in df.iterrows():
    print(row['case_submitter_id'] + " " + row["figo_stage"])

In [21]:
y = []
for file in files:
    file = file.split("/")[-1][:-4]
    count = 0
    flag = -1
    for filename in new_filenames:
        if file == filename:
            y.append(survival_info[count])
            flag = 1
        count += 1
        
    if flag == -1:
        print(file)

In [22]:
print(len(y))

90


In [23]:
train_labels = []
val_labels = []
count = 0
val_files = []
for index in range(30, 90):
    count += 1
    current_labels = []
    current_labels.append(count)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    train_labels.append(current_labels)
    
'''
for index in range(60, 90):
    count += 1
    current_labels = []
    current_labels.append(count)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    train_labels.append(current_labels)
'''
"""
for index in range(0, 27, 2):
    count += 1
    current_labels = []
    current_labels.append(count)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    train_labels.append(current_labels)
"""

count = 0
for index in range(0, 30):
    val_files.append(files[index])
    count += 1
    current_labels = []
    current_labels.append(count)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    val_labels.append(current_labels)
print(len(train_labels))
print(len(val_labels))

60
30


In [24]:
rows = []
rows.append("")
rows.append("Survival (in days)")
rows.append("Status")

In [25]:
final_train_labels = []
final_train_labels.append(rows)
for index in range(0, len(train_labels)):
    final_train_labels.append(train_labels[index])
print(len(final_train_labels))

61


In [26]:
final_val_labels = []
final_val_labels.append(rows)
for index in range(0, len(val_labels)):
    final_val_labels.append(val_labels[index])
print(len(final_val_labels))

31


In [27]:
# write csv file
with open("train_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_labels)):
        spamwriter.writerow(final_train_labels[index])

In [28]:
# write csv file
with open("val_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_labels)):
        spamwriter.writerow(final_val_labels[index])